# Equations of motion for a "zagi-like" flying wing UAV

Do imports.

In [30]:
import numpy as np
import sympy as sym
from scipy import linalg
import matplotlib.pyplot as plt
from IPython.display import display, Markdown
from scipy.signal import place_poles
from scipy.linalg import solve_continuous_are
from scipy.optimize import minimize

Define variables and parameters as symbols.

In [3]:
#
# VARIABLES
#

# Time
t = sym.Symbol('t', real=True)

# Components of position (meters)
p_x, p_y, p_z = sym.symbols('p_x, p_y, p_z', real=True)

# Yaw, pitch, and roll angles (radians)
psi, theta, phi = sym.symbols('psi, theta, phi', real=True)

# Components of linear velocity in the body frame (meters / second)
v_x, v_y, v_z = sym.symbols('v_x, v_y, v_z', real=True)

# Components of angular velocity in the body frame (radians / second)
w_x, w_y, w_z = sym.symbols('w_x, w_y, w_z', real=True)

# Elevon angles
delta_r, delta_l = sym.symbols('delta_r, delta_l', real=True)

#
# PARAMETERS
#

# Aerodynamic parameters
rho, S, c, b = sym.symbols('rho, S, c, b', real=True)
C_L_0, C_L_alpha, C_L_q, C_L_delta_e = sym.symbols('C_L_0, C_L_alpha, C_L_q, C_L_delta_e', real=True)
C_D_0, C_D_alpha, C_D_q, C_D_delta_e = sym.symbols('C_D_0, C_D_alpha, C_D_q, C_D_delta_e', real=True)
C_m_0, C_m_alpha, C_m_q, C_m_delta_e = sym.symbols('C_m_0, C_m_alpha, C_m_q, C_m_delta_e', real=True)
C_Y_0, C_Y_beta, C_Y_p, C_Y_r, C_Y_delta_a = sym.symbols('C_Y_0, C_Y_beta, C_Y_p, C_Y_r, C_Y_delta_a', real=True)
C_l_0, C_l_beta, C_l_p, C_l_r, C_l_delta_a = sym.symbols('C_l_0, C_l_beta, C_l_p, C_l_r, C_l_delta_a', real=True)
C_n_0, C_n_beta, C_n_p, C_n_r, C_n_delta_a = sym.symbols('C_n_0, C_n_beta, C_n_p, C_n_r, C_n_delta_a', real=True)
e, alpha_0, C_D_p, M = sym.symbols('e, alpha_0, C_D_p, M', real=True)
k, k_e = sym.symbols('k, k_e', real=True)

# Mass and inertia parameters
J_x, J_y, J_z, J_xz = sym.symbols('J_x, J_y, J_z, J_xz', real=True)
m, g = sym.symbols('m, g', real=True)

Define numerical values of parameters.

In [4]:
params = {
    g: 9.81,               # Gravity (m/s²)
    m: 1.56,               # Mass of the UAV (kg)
    J_x: 0.1147,           # Moment of inertia about x-axis (kg·m²) [UPDATED 02/28/2025]
    J_y: 0.0576,           # Moment of inertia about y-axis (kg·m²) [UPDATED 02/28/2025]
    J_z: 0.1712,           # Moment of inertia about z-axis (kg·m²) [UPDATED 02/28/2025]
    J_xz: 0.0015,          # Product of inertia (kg·m²)             [UPDATED 02/28/2025]

    S: 0.4696,             # Wing area (m²)
    b: 1.4224,             # Wingspan (m)
    c: 0.3302,             # Mean aerodynamic chord (m)

    rho: 1.2682,           # Air density (kg/m³)

    # Lift Coefficients
    C_L_0: 0.2,            # Lift coefficient at zero AoA
    C_L_alpha: 4.8,        # Lift curve slope (1/rad)
    C_L_q: 2.2,            # Pitch rate effect on lift (1/rad)

    # Drag Coefficients
    C_D_0: 0.02,           # Zero-lift drag coefficient
    C_D_alpha: 0.30,       # Drag change per AoA (1/rad)
    C_D_q: 0.0,            # Pitch rate effect on drag (1/rad)
    C_D_p: 0.03,           # Parasitic drag coefficient

    # Pitching Moment Coefficients
    C_m_0: -0.02,          # Pitching moment at zero AoA
    C_m_alpha: -0.6,       # Pitching moment change per AoA (1/rad)
    C_m_q: -1.8,           # Pitch rate effect on moment (1/rad)
    C_m_delta_e: -0.35,    # Effect of elevator deflection on pitching moment (1/rad)

    # Side Force Coefficients
    C_Y_0: 0.0,            # Side force at zero sideslip
    C_Y_beta: -0.08,       # Side force per sideslip angle (1/rad)
    C_Y_p: 0.0,            # Side force due to roll rate
    C_Y_r: 0.0,            # Side force due to yaw rate
    C_Y_delta_a: 0.0,      # Side force due to aileron deflection

    # Roll Moment Coefficients
    C_l_0: 0.0,            # Roll moment at zero sideslip
    C_l_beta: -0.10,       # Roll moment due to sideslip (1/rad)
    C_l_p: -0.45,          # Roll damping derivative (1/rad)
    C_l_r: 0.03,           # Roll moment due to yaw rate (1/rad)
    C_l_delta_a: 0.18,     # Aileron effect on roll (1/rad)

    # Yaw Moment Coefficients
    C_n_0: 0.0,            # Yaw moment at zero sideslip
    C_n_beta: 0.008,       # Yaw moment due to sideslip (1/rad)
    C_n_p: -0.022,         # Yaw moment due to roll rate (1/rad)
    C_n_r: -0.009,         # Yaw damping derivative (1/rad)
    C_n_delta_a: -0.004,   # Aileron effect on yaw (1/rad)

    # Control Derivatives
    C_L_delta_e: 0.30,     # Effect of elevator deflection on lift (1/rad)
    C_D_delta_e: 0.32,     # Effect of elevator deflection on drag (1/rad)

    # Efficiency Factors
    e: 0.85,               # Oswald efficiency factor
    alpha_0: 0.45,         # Zero-lift angle of attack (rad)

    # Additional Drag & Lift Coefficients
    M: 50.0,               # Sigmoid blending function parameter
    k_e: 0.01,             # Drag due to elevator deflection (empirical coefficient)
    k: 0.048               # Induced drag factor
}

Compute aerodynamic forces and torques (assuming low angles of attack, so no stall).

In [5]:
# Get airspeed, angle of attack, and angle of sideslip
V_a = sym.sqrt(v_x**2 + v_y**2 + v_z**2)
alpha = sym.atan(v_z / v_x)
beta = sym.asin(v_y / V_a)

# Convert from right and left elevon deflections to equivalent elevator and aileron deflections
delta_e = (delta_r + delta_l) / 2
delta_a = (-delta_r + delta_l) / 2

# Longitudinal aerodynamics
C_L = C_L_0 + C_L_alpha * alpha
F_lift = rho * V_a**2 * S * (C_L + C_L_q * (c / (2 * V_a)) * w_y + C_L_delta_e * delta_e) / 2
F_drag = rho * V_a**2 * S * ((C_D_0 + k * C_L**2) + C_D_q * (c / (2 * V_a)) * w_y + k_e * (C_L_delta_e * delta_e)**2) / 2
f_x, f_z = sym.Matrix([[sym.cos(alpha), -sym.sin(alpha)], [sym.sin(alpha), sym.cos(alpha)]]) @ sym.Matrix([[-F_drag], [-F_lift]])
tau_y = rho * V_a**2 * S * c * (C_m_0 + C_m_alpha * alpha + C_m_q * (c / (2 * V_a)) * w_y + C_m_delta_e * delta_e) / 2

# Lateral aerodynamics
f_y =   rho * V_a**2 * S *     (C_Y_0 + C_Y_beta * beta + C_Y_p * (b / (2 * V_a)) * w_x + C_Y_r * (b / (2 * V_a)) * w_z + C_Y_delta_a * delta_a) / 2
tau_x = rho * V_a**2 * S * b * (C_l_0 + C_l_beta * beta + C_l_p * (b / (2 * V_a)) * w_x + C_l_r * (b / (2 * V_a)) * w_z + C_l_delta_a * delta_a) / 2
tau_z = rho * V_a**2 * S * b * (C_n_0 + C_n_beta * beta + C_n_p * (b / (2 * V_a)) * w_x + C_n_r * (b / (2 * V_a)) * w_z + C_n_delta_a * delta_a) / 2

Define the linear velocity vector $v^B_{W, B}$ and the angular velocity vector $w^B_{W, B}$ in the coordinates of the body frame.

In [6]:
v_inB_ofWB = sym.Matrix([v_x, v_y, v_z])
w_inB_ofWB = sym.Matrix([w_x, w_y, w_z])

Define the moment of inertia matrix in the coordinates of the body frame.

In [7]:
J_inB = sym.Matrix([[  J_x,    0, -J_xz],
                    [    0,  J_y,     0],
                    [-J_xz,    0,   J_z]])

Define individual rotation matrices.

In [8]:
Rz = sym.Matrix([[sym.cos(psi), -sym.sin(psi), 0],
                 [sym.sin(psi), sym.cos(psi), 0],
                 [0, 0, 1]])

Ry = sym.Matrix([[sym.cos(theta), 0, sym.sin(theta)],
                 [0, 1, 0],
                 [-sym.sin(theta), 0, sym.cos(theta)]])

Rx = sym.Matrix([[1, 0, 0],
                 [0, sym.cos(phi), -sym.sin(phi)],
                 [0, sym.sin(phi), sym.cos(phi)]])

Apply sequential transformation to compute the rotation matrix that describes the orientation of the aircraft (i.e., of frame $B$ in the coordinates of frame $W$).

In [9]:
R_inW_ofB = Rz @ Ry @ Rx

Compute the matrix $N$ for which

$$\begin{bmatrix} \dot{\psi} \\ \dot{\theta} \\ \dot{\phi} \end{bmatrix} = N w_{W, B}^{B}.$$

In [10]:
# First, compute the inverse of N
Ninv = sym.Matrix.hstack((Ry * Rx).T * sym.Matrix([0, 0, 1]),
                              (Rx).T * sym.Matrix([0, 1, 0]),
                                       sym.Matrix([1, 0, 0]))

# Then, take the inverse of this result to compute N
N = sym.simplify(Ninv.inv())

Compute the total force and torque on the aicraft in the body frame (remember that $z$ is down!).

In [11]:
# Total force
f_inB = R_inW_ofB.T * sym.Matrix([0, 0, m * g]) + sym.Matrix([f_x, f_y, f_z])

# Total torque
tau_inB = sym.Matrix([tau_x, tau_y, tau_z])

Define (fully) symbolic equations of motion. These are complicated enough that we won't bother to show them.

In [12]:
f_sym = sym.Matrix.vstack(
    R_inW_ofB * v_inB_ofWB,
    N * w_inB_ofWB,
    (1 / m) * (f_inB - w_inB_ofWB.cross(m * v_inB_ofWB)),
    J_inB.inv() * (tau_inB - w_inB_ofWB.cross(J_inB * w_inB_ofWB)),
)

Substitute numerical values for each parameter. The only symbols that remain in these equations of motion are the (nonlinear) states and inputs. Again, these equations of motion are complicated, so we won't bother to show them. They have the following form:

$$
\begin{bmatrix} \dot{p}_x \\ \dot{p}_y \\ \dot{p}_z \\ \dot{\psi} \\ \dot{\theta} \\ \dot{\phi} \\ \dot{v}_x \\ \dot{v}_y \\ \dot{v}_z \\ \dot{w}_x \\ \dot{w}_y \\ \dot{w}_z \end{bmatrix}
=
f\left(
\begin{bmatrix} p_x \\ p_y \\ p_z \\ \psi \\ \theta \\ \phi \\ v_x \\ v_y \\ v_z \\ w_x \\ w_y \\ w_z \end{bmatrix},
\begin{bmatrix} \delta_r \\ \delta_l \end{bmatrix}
\right).
$$

In [13]:
f = f_sym.subs(params)

a = f[1:2]
b = f[3:14]
f_cut = sym.Matrix(a+b)

#display(f_cut)
#display(f) # Basically useless

In [67]:
# Define generalized coordinates and their derivatives
m = sym.Matrix([p_y,psi, theta, phi, v_x, v_y, v_z, w_x, w_y, w_z])
mu = sym.Matrix([p_y,psi, theta, phi, v_x, v_y, v_z, w_x, w_y, w_z, delta_r, delta_l])
u = sym.Matrix([delta_r, delta_l])

xd = f_cut
An = sym.lambdify(mu,xd.jacobian([m]))
Bn = sym.lambdify(mu,xd.jacobian([u]))

# TODO pick better equlibirum points, these are *okay* but could be better
#      p_y,psi, theta,         phi, v_x, v_y, v_z, w_x, w_y, w_z, delta_r, delta_l
A = An(0,  0,  0,              0 ,  25,   0,   -1,   0,   0,   0,   0,       0)
B = Bn(0,  0,  0,              0 ,  25,   0,   -1,   0,   0,   0,   0,       0)

#      p_y,  psi,     theta,  phi,  v_x,    v_y,    v_z,   w_x,     w_y,  w_z,  delta_r, delta_l
A = An(0.000, -0.000, 0.138, 0.000, 6.524, 0.000, 1.488, -0.000, -0.000, 0.000, -0.441, -0.441) # fall rate is too high
B = Bn(0.000, -0.000, 0.138, 0.000, 6.524, 0.000, 1.488, -0.000, -0.000, 0.000, -0.441, -0.441)

#      0.000, 0.000, 0.227, -0.000, 5.294, -0.000, 1.851, -0.000, 0.000, -0.000, -0.634, -0.634
#      0.000, 0.000, 0.007, -0.000, 9.991, -0.000, 0.766, -0.000, 0.000, -0.000, -0.188, -0.188 # this one might work
A = An(0.000, 0.000, 0.007, -0.000, 9.991, -0.000, 0.766, -0.000, 0.000, -0.000, -0.188, -0.188)
B = Bn(0.000, 0.000, 0.007, -0.000, 9.991, -0.000, 0.766, -0.000, 0.000, -0.000, -0.188, -0.188)

#      0.000, 0.000, 0.111, -0.000, 7.035, -0.001, 1.355, -0.000, 0.000, -0.000, -0.383, -0.383 # new EQ
A = An(0.000, 0.000, 0.111, -0.000, 7.035, -0.001, 1.355, -0.000, 0.000, -0.000, -0.383, -0.383)
B = Bn(0.000, 0.000, 0.111, -0.000, 7.035, -0.001, 1.355, -0.000, 0.000, -0.000, -0.383, -0.383)

#A = An(0.000, -0.000, 0.250, -0.000, 5.221, -0.000, .5, -0.000, -0.000, 0.000, -.984, -.984)
#B = Bn(0.000, -0.000, 0.250, -0.000, 5.221, -0.000, .5, -0.000, -0.000, 0.000, -.984, -.984)
display(sym.Matrix(A))
display(sym.Matrix(B))
#eq_points = sym.solve(f_cut,m)
#eq_points

Matrix([
[0.0, 7.14180169621525,               0.0,           -1.355,                  0.0,                  1.0,                  0.0,                0.0,               0.0,                0.0],
[0.0,              0.0,               0.0,              0.0,                  0.0,                  0.0,                  0.0,                0.0,               0.0,   1.00619228572814],
[0.0,              0.0,               0.0,              0.0,                  0.0,                  0.0,                  0.0,                0.0,               1.0,                0.0],
[0.0,              0.0,               0.0,              0.0,                  0.0,                  0.0,                  0.0,                1.0,               0.0,  0.111458135005104],
[0.0,              0.0, -9.74962752064922,              0.0,  -0.0648254024214451, -4.22397673468489e-5,     1.93660354582171,                0.0, -1.26105572988292,             -0.001],
[0.0,              0.0,               0.0, 9.74962752064

Matrix([
[              0.0,                0.0],
[              0.0,                0.0],
[              0.0,                0.0],
[              0.0,                0.0],
[0.281265639770187,  0.281265639770187],
[              0.0,                0.0],
[ -1.4424419460494,   -1.4424419460494],
[-17.0568912033001,   17.0568912033001],
[-15.3329768661896,  -15.3329768661896],
[0.104522591989064, -0.104522591989064]])

In [68]:
A_num = np.array(A).astype(float)
B_num = np.array(B).astype(float)

# We should take an automated approach to testing out various poles to see which works.
desired_poles = [-10,-20,-3,-4,-5,-6,-7,-8,-9,-11] # Desired pole locations

# Compute gain matrix K
K_place = place_poles(A_num, B_num, desired_poles, maxiter=1000).gain_matrix
Fn = A_num - B_num @ K_place

eig_values = np.linalg.eigvals(Fn)

print("Computed Gain Matrix K:\n", K_place.tolist())
print("Eigenvalues of F_n:", np.sort(eig_values))
print("Targeted poles    :", np.sort(desired_poles))

Computed Gain Matrix K:
 [[-1534.0411187209265, -10446.053949048179, -8.37333912597382, 4150.427910924574, 1.304678337664958, -516.2616957221518, 0.6807286075800351, 51.720716218735575, -0.7771648139736959, 8828.027988185739], [-401.1186752556395, -2824.4488462177073, -7.782240522127182, 1094.392023859124, 1.219619969947604, -135.93835372352765, 0.6497566298181775, 14.208781033753242, -0.7386541186066975, 2307.0498503933673]]
Eigenvalues of F_n: [-20.         -11.         -10.          -9.          -7.99999999
  -7.00000001  -6.          -5.          -4.          -3.        ]
Targeted poles    : [-20 -11 -10  -9  -8  -7  -6  -5  -4  -3]


C:\Users\jelly\AppData\Local\Temp\ipykernel_3148\3582211726.py:8: UserWarning: Convergence was not reached after maxiter iterations.
You asked for a tolerance of 0.001, we got 0.9999999999592722.
  K_place = place_poles(A_num, B_num, desired_poles, maxiter=1000).gain_matrix


In [78]:
def lqr(A, B, Q, R):
    P = linalg.solve_continuous_are(A, B, Q, R)
    K = linalg.inv(R) @  B.T @ P
    return K, P

Q = np.diag([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])
R = np.diag([1., 1.])

K, P = lqr(A, B, Q, R)
display(sym.Matrix(K))
display(K)


Matrix([
[-0.707080480392236, -10.8219752093133,   -3.478943491828, -5.68673648582942, 0.493813163783551, -1.37370197816552, 0.141456031474526, -0.716555724134964, -0.717357852485604, -6.02453807024472],
[ 0.707133081002651,  10.8213090906008, -3.47869456624136,  5.68534237349051, 0.493814126490846,  1.37360048461444, 0.141481427340814,  0.716530036643642, -0.717349030223311,  6.02198822996857]])

array([[ -0.70708048, -10.82197521,  -3.47894349,  -5.68673649,
          0.49381316,  -1.37370198,   0.14145603,  -0.71655572,
         -0.71735785,  -6.02453807],
       [  0.70713308,  10.82130909,  -3.47869457,   5.68534237,
          0.49381413,   1.37360048,   0.14148143,   0.71653004,
         -0.71734903,   6.02198823]])

In [18]:
from sympy import *
from scipy.optimize import root

p_ye =0
w_ze = 0


f = f_sym.subs(params)

a = f[1:2]
b = f[3:14]
fnew = Matrix(a+b)

f_num = sym.lambdify([p_y,psi,theta,phi,v_x,v_y,v_z,w_x,w_y,w_z,delta_r,delta_l],fnew)

f_num_root = lambda x: f_num(p_ye, x[0],x[1],x[2],x[3],x[4],x[5],x[6], x[7], w_ze, x[8],x[9]).flatten()

#      p_y,  psi,   theta,  phi,  v_x,    v_y,    v_z,   w_x,     w_y,  w_z,  delta_r, delta_l
x0 =   [     0.000,0.000,0.000,5.000, 0.000,  0.500,   0.000,  0.000,         -0.060, -0.060]

sol = root(f_num_root,x0)
#print(f"{sol.x[0]:.6f}")
psi_e = sol.x[0]
theta_e = sol.x[1]
phi_e = sol.x[2]
vx_e = sol.x[3]
vy_e = sol.x[4]
vz_e = sol.x[5]
wx_e = sol.x[6]
wy_e = sol.x[7]
delta_re = sol.x[8]
delta_le =sol.x[9]

#print(f_num(p_ye, psi_e, theta_e, phi_e, vx_e, vy_e, vz_e, wx_e, wy_e, w_ze, delta_re, delta_le))

#print(p_ye, psi_e, theta_e, phi_e, vx_e, vy_e, vz_e, wx_e, wy_e, w_ze, delta_re, delta_le)
#new cell
A_num = sym.lambdify([p_y,psi,theta,phi,v_x,v_y,v_z,w_x,w_y,w_z,delta_r,delta_l],fnew.jacobian([p_y,psi,theta,phi,v_x,v_y,v_z,w_x,w_y,w_z]))
B_num = sym.lambdify([p_y,psi,theta,phi,v_x,v_y,v_z,w_x,w_y,w_z,delta_r,delta_l],fnew.jacobian([delta_r,delta_l]))

A = A_num(p_ye, psi_e, theta_e, phi_e, vx_e, vy_e, vz_e, wx_e, wy_e, w_ze, delta_re, delta_le)
B = B_num(p_ye, psi_e, theta_e, phi_e, vx_e, vy_e, vz_e, wx_e, wy_e, w_ze, delta_re, delta_le)

#Matrix(B)
print(f"{p_ye:.3f}, {psi_e:.3f}, {theta_e:.3f}, {phi_e:.3f}, {vx_e:.3f}, {vy_e:.3f}, {vz_e:.3f}, {wx_e:.3f}, {wy_e:.3f}, {w_ze:.3f}, {delta_re:.3f}, {delta_le:.3f}")

# Substitute values
A_num = np.array(A).astype(float)
B_num = np.array(B).astype(float)

# We should take an automated approach to testing out various poles to see which works.
desired_poles = [-10,-20,-3,-4,-5,-6,-7,-8,-9,-11] # Desired pole locations

# Compute gain matrix K
K_place = place_poles(A_num, B_num, desired_poles, maxiter=1000).gain_matrix

# Compute F_n = A - BK
Fn = A_num - B_num @ K_place

# Compute eigenvalues of F_n
eig_values = np.linalg.eigvals(Fn)

# Output results
print("Computed Gain Matrix K:\n", K_place.tolist())
print("Eigenvalues of F_n:", np.sort(eig_values))
print("Targeted poles    :", np.sort(desired_poles))

0.000, -0.000, 0.650, -0.000, 2.221, -0.000, 2.746, -0.000, -0.000, 0.000, -1.584, -1.584
Computed Gain Matrix K:
 [[-5287.578247199959, -30547.139566992715, -40.098364741791215, 22897.501158002604, -0.13196412004727975, -2239.45072598521, 5.686911247546208, 517.6733543313546, -3.6287795416700144, 85450.05489535719], [13610.258120120741, 66166.7846406815, -32.17185441516256, -59782.3024328155, -0.5595109754258787, 5340.08643500163, 3.8295428031243013, -1395.405655639092, -3.257494772187899, -228645.7830837267]]
Eigenvalues of F_n: [-19.99999998 -11.00000016 -10.00000013  -8.99999984  -7.99999962
  -6.99999973  -6.00000089  -4.99999976  -3.99999985  -3.00000005]
Targeted poles    : [-20 -11 -10  -9  -8  -7  -6  -5  -4  -3]


C:\Users\jelly\AppData\Local\Temp\ipykernel_3148\2413736680.py:55: UserWarning: Convergence was not reached after maxiter iterations.
You asked for a tolerance of 0.001, we got 0.9999999999999962.
  K_place = place_poles(A_num, B_num, desired_poles, maxiter=1000).gain_matrix


In [65]:
# Lambda function to evaluate f_reduced
f_num = sym.lambdify(mu, f_cut)

# Lambda function to evalaute the cost function for minimization (i.e., the norm squared of f_reduced)
f_num_for_minimize = lambda x: np.linalg.norm(f_num(*x).flatten())**2

#      p_y,psi, theta,         phi, v_x, v_y, v_z, w_x, w_y, w_z, delta_r, delta_l
x0 =   [0,  0,  0.1,              0 ,  7,   0,   .2,   0,   0,   0,   -.1,       -.1]

# Find minimum ("tol" is a tolerance that says how accurate you want the solution)
sol = minimize(f_num_for_minimize, x0, tol=1e-4)

# Show solution
print(sol)
print('')
print(", ".join(f"{val:.3f}" for val in sol.x))


  message: Optimization terminated successfully.
  success: True
   status: 0
      fun: 4.260123524098284e-09
        x: [ 0.000e+00  9.498e-05  1.107e-01 -7.015e-06  7.035e+00
            -6.878e-04  1.355e+00 -2.514e-05  1.766e-06 -3.847e-06
            -3.832e-01 -3.834e-01]
      nit: 18
      jac: [ 0.000e+00 -4.948e-07  8.896e-07 -1.181e-06  5.438e-07
            -1.091e-05  1.061e-05  2.344e-06  7.845e-07  1.644e-06
             2.524e-05  7.411e-06]
 hess_inv: [[ 1.000e+00  0.000e+00 ...  0.000e+00  0.000e+00]
            [ 0.000e+00  3.675e-02 ...  1.710e-02 -1.410e-02]
            ...
            [ 0.000e+00  1.710e-02 ...  1.157e-01  4.779e-02]
            [ 0.000e+00 -1.410e-02 ...  4.779e-02  1.132e-01]]
     nfev: 351
     njev: 27

0.000, 0.000, 0.111, -0.000, 7.035, -0.001, 1.355, -0.000, 0.000, -0.000, -0.383, -0.383
